### PDF-analys (Text vs. OCR) - för att avgöra om en pdf är textbaserad eller skannade bilder (som kommer kräva OCR)

### Importera bibliotek

In [1]:
import pdfplumber
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm  # För en snygg progress bar i notebooken

### Sätt upp sökvägar

In [2]:
BASE_DIR = Path(r"C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden")
DATA_DIR = BASE_DIR / "data"

# VIKTIGT: Vi pekar nu på den RIKTIGA, städade mappen
CLEAN_DATA_DIR = DATA_DIR / "01_raw"
# TEST_DATA_DIR = DATA_DIR / "00_test"

# Mapp för att spara bearbetade filer (enligt filstrukturen)
PROCESSED_DIR = DATA_DIR / "02_processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Filen där vi sparar vår analys-rapport
ANALYSIS_RESULT_FILE = PROCESSED_DIR / "pdf_analysis_report.csv"

### Hitta NYA PDF-filer att analysera

In [3]:
# Vi laddar den befintliga rapporten för att se vilka filer vi redan har analyserat.

# Läs den BEFINTLIGA rapporten, om den finns
try:
    df_existing = pd.read_csv(ANALYSIS_RESULT_FILE)
    # Skapa en 'set' (uppsättning) av alla filvägar vi redan känner till
    # Vi måste normalisera sökvägarna för att jämförelsen ska fungera
    seen_files = set(df_existing['full_path'].apply(lambda x: str(Path(x))))
    print(f"Hittade en befintlig rapport med {len(seen_files)} analyserade filer.")
except FileNotFoundError:
    print("Ingen befintlig rapport hittades. Startar en ny analys.")
    df_existing = pd.DataFrame(columns=["full_path", "status", "chars_page_1", "total_pages", "filename"]) # Skapa tom
    seen_files = set()

# Hämta ALLA PDF-filer som finns på disk NU
print(f"Söker efter ALLA PDF-filer i: {CLEAN_DATA_DIR}...")
all_pdf_files = list(CLEAN_DATA_DIR.rglob("*.pdf"))
print(f"Hittade totalt {len(all_pdf_files)} PDF-filer på disken.")

# Jämför och hitta de som är NYA
files_to_analyze = []
for file_path in all_pdf_files:
    # Normalisera sökvägen även här
    normalized_path = str(file_path)
    if normalized_path not in seen_files:
        files_to_analyze.append(file_path)

print(f"\n--- Analys-sammanfattning ---")
print(f"Totalt antal filer på disk: {len(all_pdf_files)}")
print(f"Filer redan analyserade:   {len(seen_files)}")
print(f"NYA filer att analysera:    {len(files_to_analyze)}")

# Kontroll: Vi behöver också se om filer har tagits bort
existing_paths_on_disk = set(str(p) for p in all_pdf_files)
df_existing = df_existing[df_existing['full_path'].apply(lambda x: str(Path(x)) in existing_paths_on_disk)]
print(f"Rensade bort {len(seen_files) - len(df_existing)} borttagna filer från rapporten.")

Hittade en befintlig rapport med 3617 analyserade filer.
Söker efter ALLA PDF-filer i: C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden\data\01_raw...
Hittade totalt 4502 PDF-filer på disken.

--- Analys-sammanfattning ---
Totalt antal filer på disk: 4502
Filer redan analyserade:   3617
NYA filer att analysera:    885
Rensade bort 0 borttagna filer från rapporten.


### Hämtar alla pdf-filer

In [4]:
# print(f"Söker efter PDF-filer i: {TEST_DATA_DIR}...")
# pdf_files = list(TEST_DATA_DIR.rglob("*.pdf"))
# print(f"Hittade {len(pdf_files)} PDF-filer att analysera.")

print(f"Söker efter PDF-filer i: {CLEAN_DATA_DIR}...")
pdf_files = list(CLEAN_DATA_DIR.rglob("*.pdf"))
print(f"Hittade {len(pdf_files)} PDF-filer att analysera.")

Söker efter PDF-filer i: C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden\data\01_raw...
Hittade 4502 PDF-filer att analysera.


#### Skapa Analysfunktionen

In [5]:
# Den här funktionen öppnar en PDF, försöker extrahera text från *första sidan*, och räknar antalet tecken.

# **Vår logik (heuristik):**
# * Om första sidan har > 50 tecken, antar vi att den är **textbaserad**.
# * Om första sidan har < 50 tecken, är den en **OCR-kandidat**.

def analyze_pdf_type(file_path: Path) -> dict:
    """
    Analyserar en enskild PDF och returnerar dess typ.
    """
    try:
        with pdfplumber.open(file_path) as pdf:
            
            # Kontrollera om PDF:en har några sidor alls
            if not pdf.pages:
                return {"status": "error_no_pages", "chars_page_1": 0, "total_pages": 0}
            
            # Hämta första sidan
            first_page = pdf.pages[0]
            
            # Försök extrahera text
            text = first_page.extract_text()
            
            # Räkna antalet "rena" tecken
            char_count = 0
            if text:
                char_count = len(text.strip())
            
            total_pages = len(pdf.pages)

            if char_count > 50:
                # Troligen en text-baserad PDF
                return {"status": "text_based", "chars_page_1": char_count, "total_pages": total_pages}
            else:
                # Troligen en skannad bild (eller en PDF med bara bilder)
                return {"status": "ocr_candidate", "chars_page_1": char_count, "total_pages": total_pages}

    except Exception as e:
        # Hantera korrupta eller lösenordsskyddade filer
        return {"status": f"error_{type(e).__name__}", "chars_page_1": 0, "total_pages": 0}

### Kör analysfunktionen

In [6]:
# Nu loopar bara igenom nya filer (`files_to_analyze`) och analyserar dem

analysis_results = []

new_analysis_results = [] # Nytt namn för tydlighet

if files_to_analyze: # Kör bara om det finns något att göra
    print(f"Startar analys av {len(files_to_analyze)} nya filer...")
    # tqdm ger oss en fin progress bar
    for file in tqdm(files_to_analyze, desc="Analyserar NYA PDF-filer"):
        result = analyze_pdf_type(file)
        result["full_path"] = str(file) # Lägg till sökvägen i resultatet
        result["filename"] = file.name
        new_analysis_results.append(result)

    print("Analys av nya filer klar.")
else:
    print("Inga nya filer att analysera. Allt är uppdaterat.")

Startar analys av 885 nya filer...


Analyserar NYA PDF-filer:   0%|          | 0/885 [00:00<?, ?it/s]

Analys av nya filer klar.


### Granska resultat och spara till fil

In [7]:
# Vi konverterar vår lista med resultat till en Pandas DataFrame och kombinerar nya ihop nya och gamla resultat.
# Detta gör det superlätt att filtrera, sortera och förstå vad vi har.

# Skapa en DataFrame av de NYA resultaten
df_new_analysis = pd.DataFrame(new_analysis_results)

# Kombinera den gamla (df_existing) med den nya (df_new_analysis)
df_combined = pd.concat([df_existing, df_new_analysis], ignore_index=True)

# Visa en sammanfattning av den TOTALA rapporten
print("\n--- Sammanfattning av TOTAL PDF-analys (Gamla + Nya) ---")
if not df_combined.empty:
    status_counts = df_combined['status'].value_counts()
    print(status_counts)
    print("\n--- Exempel på data (från nya filer om några fanns) ---")
    if not df_new_analysis.empty:
        print(df_new_analysis.head())
    else:
        print("Inga nya filer analyserades.")
else:
    print("Rapporten är tom.")

# Spara den nya, kombinerade rapporten
df_combined.to_csv(ANALYSIS_RESULT_FILE, index=False, encoding='utf-8-sig')
print(f"\nFullständig, uppdaterad analysrapport sparad till:\n{ANALYSIS_RESULT_FILE}")


--- Sammanfattning av TOTAL PDF-analys (Gamla + Nya) ---
status
text_based       4372
ocr_candidate     130
Name: count, dtype: int64

--- Exempel på data (från nya filer om några fanns) ---
       status  chars_page_1  total_pages  \
0  text_based           769            8   
1  text_based           963           22   
2  text_based          1110           33   
3  text_based           900           10   
4  text_based           726           20   

                                           full_path  \
0  C:\Users\Dator\Documents\Data_Science\11_Exame...   
1  C:\Users\Dator\Documents\Data_Science\11_Exame...   
2  C:\Users\Dator\Documents\Data_Science\11_Exame...   
3  C:\Users\Dator\Documents\Data_Science\11_Exame...   
4  C:\Users\Dator\Documents\Data_Science\11_Exame...   

                                            filename  
0      FÖRVR_2010-10-22_6720-10_bygglov och 12_6.PDF  
1  MMD_2023-03-21_M3143-22_föreläggande enligt KM...  
2  MMD_2024-11-07_M4815-23_tidsrestriktio

### Hitta en exempelfil som är en OCR-kandidat

In [8]:
# ## Spot-check av en "OCR-kandidat" (Bonus)
#
# Om du vill *bevisa* att en fil är skannad, behöver vi
# biblioteken `pytesseract` och `pdf2image`.
#
# **OBS:** Detta kräver extern installation!
# 1. `pip install pytesseract pdf2image`
# 2. Du MÅSTE också installera Googles Tesseract-motor OCH Poppler.
#    (På Windows är detta enklast via: https://github.com/UB-Mannheim/tesseract/wiki
#     och http://blog.alivate.com.au/poppler-in-windows/)
#
# **Detta är avancerat, så vi kan vänta med det.** Men det är
# här du skulle bekräfta din hypotes.


# Hitta en exempelfil som är en OCR-kandidat från den KOMBINERADE rapporten
ocr_files = df_combined[df_combined['status'] == 'ocr_candidate'] # Ändrad från df_analysis

if not ocr_files.empty:
    print("\n--- Exempel på filer som troligen kräver OCR ---")
    print(ocr_files[['filename', 'chars_page_1']].head())
else:
    print("\nGratulerar! Inga uppenbara OCR-kandidater hittades.")


--- Exempel på filer som troligen kräver OCR ---
                                              filename  chars_page_1
95                          Dnr 502-2025 (ansökan).pdf             0
158                            22378-2025 OSL 21-1.pdf             0
413  11635-2025 Anmälan för samråd om solcellspark(...            12
470                   2076-2025 anmälan och beslut.pdf             0
538               Naturvärdesinventering(30759500).pdf            50
